In [5]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
import shutil
from tqdm import tqdm_notebook

Substitude all "holing" in this jupyter notebook to your username

In [18]:
voc_label_path = "/home/holing/voc/VOCdevkit/VOC2012/Annotations/"
classes = ["person", "bird","cat","cow","dog","horse","sheep",'aeroplane','bicycle','boat',"bus","car", \
           "motorbike","train",'bottle','chair','dinning table','potted plant',"sofa",'tvmonitor']

In [29]:
sets=[('2012', 'train'),('2012', 'val'),('2012','test')]

convert label from VOC to YOLO

In [9]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 
    y = (box[2] + box[3])/2.0 
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [27]:
def convert_annotation(year, image_id):
    in_file = open('/home/holing/voc/VOCdevkit/VOC%s/Annotations/%s.xml'%(year, image_id))
    out_file = open('./data/VOC/labels/%s.txt'%( image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls not in classes:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')


In [10]:
wd = getcwd()

In [35]:
#for year, image_set in sets:
year = '2012'
image_set = 'train'
if not os.path.exists('./data/VOC/labels/'):
    os.makedirs('./data/VOC/labels/')
if not os.path.exists('./data/VOC/images/'):
    os.makedirs('./data/VOC/images/')
image_ids = open('/home/holing/voc/VOCdevkit/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
list_file = open('./data/%s_%s.txt'%(year, image_set), 'w')
for image_id in tqdm_notebook(image_ids):
    old_name = '/home/holing/voc/VOCdevkit/VOC%s/JPEGImages/%s.jpg'%( year, image_id)
    new_name = './data/VOC/images/%s.jpg'%( image_id)
    shutil.copyfile(old_name, new_name)
    list_file.write(new_name+'\n')
    convert_annotation(year, image_id)
list_file.close()


 64%|██████▍   | 3672/5717 [00:41<00:18, 110.85it/s]


100%|██████████| 5717/5717 [01:14<00:00, 76.36it/s]

split val into val and test

In [36]:
year = '2012'
image_set = 'val'
image_ids = open('/home/holing/voc/VOCdevkit/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
list_file = open('./data/%s_%s.txt'%(year, image_set), 'w')
for image_id in tqdm(image_ids[:2000]):
    old_name = '/home/holing/voc/VOCdevkit/VOC%s/JPEGImages/%s.jpg'%( year, image_id)
    new_name = './data/VOC/images/%s.jpg'%( image_id)
    shutil.copyfile(old_name, new_name)
    list_file.write(new_name +'\n')
    convert_annotation(year, image_id)
list_file.close()

image_set = 'test'
list_file = open('./data/%s_%s.txt'%(year, image_set), 'w')
for image_id in tqdm_notebook(image_ids[2000:]):
    old_name = '/home/holing/voc/VOCdevkit/VOC%s/JPEGImages/%s.jpg'%( year, image_id)
    new_name = './data/VOC/images/%s.jpg'%( image_id)
    shutil.copyfile(old_name, new_name)
    list_file.write(new_name+'\n')
    convert_annotation(year, image_id)
list_file.close()


 44%|████▍     | 1698/3823 [00:42<00:18, 115.41it/s]


100%|██████████| 3823/3823 [01:03<00:00, 60.21it/s] 